In [1383]:
a = [1,2,3]
b= [[4],[5],[6]]

b+=a
print(b)

[[4], [5], [6], 1, 2, 3]


In [1384]:
def print_vector(vec):
    for x in vec:
        print(x, end=" ")
    print()  # final newline


In [1385]:
def andify(vector_of_ands): # vector_of_ands is a vector of vectors
    v = []
    for i in range(len(vector_of_ands)):
        v.append(vector_of_ands[i])
        if i != len(vector_of_ands)-1:
            v.append("AND")
    return v

def orify(vector_of_ors): # vector_of_ands is a vector of vectors
    v = []
    for i in range(len(vector_of_ors)):
        v.append(vector_of_ors[i])
        if i != len(vector_of_ors)-1:
            v.append("OR")
    return v

In [1386]:
def K(b, w, t): # Is 1 if you learn word w reading book b at time t
    # return [f"K({b},{w},{t})"]
    #delete later?# global v
    v_prime = []
    v_prime.append("(")

    # Now s_d goes from 0 to len(B[b]) - 1
    for s_d in range(len(B[b])):

        if w in B[b][s_d]:
            if len(B[b][s_d]) == 1: 
                # w is alone in s_d, so we learned w in this book
                v_prime.clear()
                #delete later?# v += [1]
                # So, we can return a 1, cause we will learn w in this book b
                return [1]

            v_prime.append("(")

            for omega in B[b][s_d]:
                if omega == w:
                    continue

                v_prime += k(b, s_d, omega, t)
                v_prime.append("AND")
            if v_prime[-1] == "AND": del v_prime[-1]

            v_prime.append(")")
            v_prime.append("OR")
    if v_prime[-1] == "OR": del v_prime[-1]

    v_prime.append(")")

    if v_prime == ["(",")"]:
        # we never found w in book b
        v_prime.clear()
        # So, we can return a 0, cause we won't learn w in this book b
        return [0]  
    
    #delete later?# v.extend(v_prime)
    
    return v_prime





# def K(b, w, t): # Is 1 if you learn word w reading book b at time t
#     # return [f"K({b},{w},{t})"]
#     v_prime = []

#     # Now s_d goes from 0 to len(B[b]) - 1
#     for s_d in range(len(B[b])):

#         if w in B[b][s_d]:
#             if len(B[b][s_d]) == 1: 
#                 # w is alone in s_d, so we learned w in this book
#                 # So, we can return a 1, cause we will learn w in this book b
#                 return 1

#             vector_of_ands = []
#             for omega in B[b][s_d]:
#                 if omega == w:
#                     continue

#                 vector_of_ands += [k(b, s_d, omega, t)]

#                 # v_prime += k(b, s_d, omega, t)
#                 # v_prime.append("AND")
#             vector_of_ands = andify(vector_of_ands)
#             v_prime += [vector_of_ands]

#     if v_prime == []:
#         # we never found w in book b
#         # So, we can return a 0, cause we won't learn w in this book b
#         return 0

#     v_prime = orify(v_prime)


#     return v_prime




In [1387]:
# def l(w, t):
#     v.append("(")
#     if t == 0:
#         for b in range(len(B)):
#             if w in B[b][0]:
#                 k(b, 0, w, t)
#                 v.append("OR")

#     else:


#     v.append(")")

In [1388]:
def k(b, s_d, omega, t):

    # APPARENTLY THIS CANT BE A ZERO, see which cases it should be and solve it


    # return [f"k({b},{s_d},{omega},{t})"]
    # v.append(f"k({b},{s_d},{omega},{t})")
    #v.append(f"\"know word {omega} at b_{b}[{s_d}] at t={t}\"")
    
    if len(B[b][s_d]) == 1 and B[b][s_d][0] == omega:
        # omega is the only word in sentence s_d of book b
        # So, we learn omega reading book b at s_d
        return [1]
    

    
    v_prime = []
    if s_d == 0: 
        # if first sentence, we can only look back
        # if we cant look back (t=0) we just dont know omega
        return (l[omega][t-1] if t!=0 else [0])
        # return ([f"l[{omega}][{t-1}]"] if t!=0 else [0])

    v_prime.append("(")

    v_prime += k(b, s_d-1, omega, t) + ["OR"]

    # DEBUG OPEN
    # print(f"k({b}, {s_d-1}, {omega}, {t}) OR")
    # print(f"    = ", end="")
    # print_vector(k(b, s_d-1, omega, t))
    # DEBUG CLOSE

    if omega in B[b][s_d]:
        v_prime.append("(")
        for w in B[b][s_d]:
            if w == omega:
                continue

            v_prime += k(b, s_d-1, w, t) + ["AND"]
            
            # DEBUG OPEN
            # print(f"k({b}, {s_d-1}, {w}, {t}) AND")
            # print(f"    = ", end="")
            # print_vector(k(b, s_d-1, w, t))
            # DEBUG CLOSE

        if v_prime[-1] == "AND": del v_prime[-1]

        v_prime.append(")")
    
    if v_prime[-1] == "OR": del v_prime[-1]

    v_prime.append(")")

    return v_prime


In [1389]:
def clean_constants(v_prime): # by ChatGPT # removes 0 OR or OR 0 and 1 AND or AND 1
    v = v_prime.copy()
    i = 0
    while i < len(v) - 2:
        a, op, b = v[i], v[i+1], v[i+2]

        # Case: 0 OR X → X
        if a == 0 and op == "OR":
            del v[i:i+2]   # remove 0, "OR"
            continue

        # Case: X OR 0 → X
        if op == "OR" and b == 0:
            del v[i+1:i+3]  # remove "OR", 0
            continue

        # Case: 1 AND X → X
        if a == 1 and op == "AND":
            del v[i:i+2]   # remove 1, "AND"
            continue

        # Case: X AND 1 → X
        if op == "AND" and b == 1:
            del v[i+1:i+3]  # remove "AND", 1
            continue

        i += 1
    return v

def collapse_parentheses(v_prime): # by Gemini # turns whole exp into 1 if OR 1 and into 0 if AND 0
    """
    Collapses parenthetical groups in a boolean vector if they match
    short-circuit conditions (AND 0, OR 1) at the top level of the group.
    
    Args:
        v_prime (list): A list of strings and integers (e.g., ['(', 'A', 'AND', 0, ')'])
    
    Returns:
        list: The modified list with short-circuits collapsed.
    """
    v = v_prime.copy()
    
    # We use a while loop because we modify the list length in place
    i = 0
    while i < len(v):
        if v[i] == "(":
            start = i
            
            # 1. Find the MATCHING closing parenthesis
            # We cannot just use .index(")") because that might catch a nested one.
            balance = 1
            end = -1
            for k in range(i + 1, len(v)):
                if v[k] == "(":
                    balance += 1
                elif v[k] == ")":
                    balance -= 1
                
                if balance == 0:
                    end = k
                    break
            
            # If no matching paren found (malformed), just move on
            if end == -1:
                i += 1
                continue

            # 2. Scan INSIDE the identified range [start+1 : end]
            # We look for short circuits, but we must SKIP nested parentheses
            j = start + 1
            found_short_circuit = None # Can be 0 or 1
            
            while j < end:
                token = v[j]
                
                # If we hit a nested open paren, skip this entire nested block
                if token == "(":
                    nested_balance = 1
                    while j < end and nested_balance > 0:
                        j += 1
                        if v[j] == "(": nested_balance += 1
                        elif v[j] == ")": nested_balance -= 1
                    # j is now at the closing paren of the nested group
                    j += 1
                    continue

                # CHECK FOR: AND 0
                # We check if token is 0, and if it has an AND neighbor
                if token == 0:
                    prev_is_and = (j > start + 1 and v[j-1] == "AND")
                    next_is_and = (j + 1 < end and v[j+1] == "AND")
                    
                    if prev_is_and or next_is_and:
                        found_short_circuit = 0
                        break

                # CHECK FOR: OR 1
                # We check if token is 1, and if it has an OR neighbor
                if token == 1:
                    prev_is_or = (j > start + 1 and v[j-1] == "OR")
                    next_is_or = (j + 1 < end and v[j+1] == "OR")
                    
                    if prev_is_or or next_is_or:
                        found_short_circuit = 1
                        break
                
                j += 1

            # 3. Apply Collapse if found
            if found_short_circuit is not None:
                # Replace everything from start to end (inclusive) with the result
                v[start:end+1] = [found_short_circuit]
                # Reset cursor to start to re-evaluate surrounding context if needed
                # (e.g., ( (A) AND 0 ) AND 0 might need two passes, checking from start is safest)
                i = 0 
            else:
                i += 1
        else:
            i += 1
            
    return v

def remove_redundant_nested_parentheses(v_prime): # by ChatGPT # turns ( ( A ) ) into ( A )
    v = v_prime.copy()
    """
    In-place: collapse only exact redundant nested parentheses of the form
      ( ( ... ) )
    into
      ( ... )

    Safety: only removes when the inner '(''s matching ')' is immediately
    followed by the outer ')'. This avoids removing '(' in cases like
      ( ( A OR B ) AND C )
    which must be preserved.
    """
    changed = True
    while changed:
        changed = False
        i = 0
        while i < len(v) - 1:
            # look for "( ("
            if v[i] == "(" and v[i+1] == "(":
                # find matching close for the inner '(' at position i+1
                depth = 0
                j = i + 1
                matched = -1
                while j < len(v):
                    if v[j] == "(":
                        depth += 1
                    elif v[j] == ")":
                        depth -= 1
                        if depth == 0:
                            matched = j
                            break
                    j += 1

                # matched is index of inner ')' if found
                # check that the token right after inner ')' is an outer ')'
                if matched != -1 and matched + 1 < len(v) and v[matched + 1] == ")":
                    # remove the outermost redundant pair:
                    # delete v[i] (the first '(') and v[matched + 1] (the corresponding outer ')')
                    # delete the later index first to avoid shifting problem
                    del v[matched + 1]
                    del v[i]
                    changed = True
                    # after deletion, stay at same i to re-check nested chains e.g. (((X))) 
                    continue
                else:
                    # not a simple redundant nested pair; advance
                    i += 1
            else:
                i += 1
    return v

def remove_redundant_nested_parentheses2(v_prime): # function that makes ( 0 ) into 0 and ( 1 ) into 1
    v = v_prime.copy()
    i = 0
    while i < len(v) - 2:
        if v[i] == "(" and v[i+2] == ")":
            if v[i+1] == 0 or v[i+1] == 1:
                # remove the parentheses
                v[i:i+3] = [v[i+1]]
                continue
        i += 1
    return v

def clean_andor_parentheses(v): # function that cleans things like AND ) and makes it just be )
    v_prime = v.copy()
    i = 0
    
    # Loop until the second to last element (since we check i+1)
    while i < len(v_prime) - 1:
        
        current_token = v_prime[i]
        next_token = v_prime[i+1]
        
        if current_token in ("AND", "OR") and next_token == ")":
            del v_prime[i]
            # Do not advance i; check the new token at this index

        elif current_token == "(" and next_token in ("AND", "OR"):
            del v_prime[i+1]
            # Do not advance i; check if another operator slid into place
            
        else:
            i += 1
            
    return v_prime
    

def clean_vector(v):
    v_prime = v.copy()
    
    while True: # repeat cleaning until no changes
        comparison_vector = v_prime.copy()
        
        v_prime = clean_andor_parentheses(v_prime)
        v_prime = remove_redundant_nested_parentheses2(v_prime)
        v_prime = clean_constants(v_prime)
        v_prime = collapse_parentheses(v_prime)
        v_prime = remove_redundant_nested_parentheses(v_prime)

        if comparison_vector == v_prime:
            break
    return v_prime


In [1390]:
# v = []
# v += ["("] + ["C"] + ["OR"] + ["("] + ["("] + ["A"] + ["AND"] + [0] + [")"] + ["AND"] + ["B"] + [")"] + [")"]

# print_vector(v)
# v = collapse_parentheses(v)
# print_vector(v)

In [ ]:
# Books
#  Creating the Modelling

v = [] # vector storing the ending expression

B = [ 
      [ [1] ] ,                          # b0
      [ [0,1] ] ,                        # b1
      [ [0,3], [2], [4,3,5], [4,0] ] ,   # b2
      [ [0,3], [2], [4,3,5], [4,0] ] ,   # b3
    ]

# B = [ # good example for testing rereads
#       [ [1,2,3,4],[1,2,3],[1,2],[1] ] ,                          # b0
#       [ [5] ] ,                        # b1
#       [ [0] ] ,   # b2
#       [ [0] ] ,   # b3
#     ]

n_of_unknown_words = 6 #max number in B - 1

n_of_books_to_read = 4 # At most the amount of books; max value t+1 can have.

n_of_books = len(B)
# L = 

In [1392]:
# See later (full_build())
    # def full_build():
    #     for b in range(len(B)):
    #         for s_d in range(len(B[b])):
    #             for w in B[b][s_d]:
    #                 for t in range(n_of_books_to_read):

    #                     L[][][]    
                        
                    
                    


    # L = [w][t][b] of 
    #   n° of words BY
    #   n° of books to read (can be less than amount of books)) BY
    #   n° of books

    # L = [[[[] for b in range(len(B))] for t in range(n_of_books_to_read)] for w in range(n_of_unknown_words)]

    # inside L[w][t][b] there will be K(b,w,t)
    # in each l[w][t] we will do OR between all b

In [1393]:
l = [[[] for t in range(n_of_books_to_read)] for w in range(n_of_unknown_words)]

for t in range(n_of_books_to_read):
    for w in range(n_of_unknown_words):
        l[w][t].append("(")

        if t != 0:
            # l[w][t].extend(["(", f"l[{w}][{t-1}]", ")", "OR"])
            l[w][t] += l[w][t-1] + ["OR"]
        
        for b in range(n_of_books):
            big_k = K(b, w, t)
            if big_k == [0]: # If w wasn't in book b, we can skip book b
                continue

            l[w][t].extend(["(", "(", f"x[{b}]", "==", t, ")", "AND"])
            l[w][t] += big_k
            l[w][t].extend([")","OR"])
        if l[w][t][-1] == "OR": del l[w][t][-1]
        
        l[w][t].append(")")
        l[w][t] = clean_vector(l[w][t])

In [1394]:
for t in range(n_of_books_to_read):
    for w in range(n_of_unknown_words):
        print(f"l[{w}][{t}]:")
        print("     ",end="")
        print_vector(l[w][t])
        print()

# print_vector(l[3][1])

l[0][0]:
     ( ( x[2] == 0 ) OR ( x[3] == 0 ) ) 

l[1][0]:
     ( x[0] == 0 ) 

l[2][0]:
     0 

l[3][0]:
     0 

l[4][0]:
     0 

l[5][0]:
     ( x[1] == 0 ) 

l[0][1]:
     ( ( ( x[2] == 0 ) OR ( x[3] == 0 ) ) OR ( x[2] == 1 ) OR ( x[3] == 1 ) ) 

l[1][1]:
     ( ( x[0] == 0 ) OR ( x[0] == 1 ) ) 

l[2][1]:
     ( ( x[0] == 1 ) AND ( x[0] == 0 ) ) 

l[3][1]:
     0 

l[4][1]:
     0 

l[5][1]:
     ( ( x[1] == 0 ) OR ( x[1] == 1 ) ) 

l[0][2]:
     ( ( ( ( x[2] == 0 ) OR ( x[3] == 0 ) ) OR ( x[2] == 1 ) OR ( x[3] == 1 ) ) OR ( x[2] == 2 ) OR ( x[3] == 2 ) ) 

l[1][2]:
     ( ( ( x[0] == 0 ) OR ( x[0] == 1 ) ) OR ( x[0] == 2 ) ) 

l[2][2]:
     ( ( ( x[0] == 1 ) AND ( x[0] == 0 ) ) OR ( ( x[0] == 2 ) AND ( ( ( ( x[0] == 0 ) OR ( x[0] == 1 ) ) AND ( ( ( x[0] == 0 ) OR ( x[0] == 1 ) ) AND ( ( x[0] == 1 ) AND ( x[0] == 0 ) ) ) ) OR ( ( ( x[0] == 0 ) OR ( x[0] == 1 ) ) OR ( ( x[0] == 1 ) AND ( x[0] == 0 ) ) ) ) ) ) 

l[3][2]:
     ( ( x[0] == 2 ) AND ( ( ( x[0] == 0 ) OR ( x[0] == 1 ) 

In [1395]:
l[0][0]:
     0 

l[1][0]:
     ( x[0] == 0 ) 

l[2][0]:
     ( ( x[2] == 0 ) OR ( x[3] == 0 ) ) 

l[3][0]:
     0 

l[4][0]:
     0 

l[5][0]:
     0 

l[0][1]:
     ( ( l[0][0] ) OR ( ( x[1] == 1 ) AND ( l[1][0] ) ) OR ( ( x[2] == 1 ) AND ( ( l[3][0] ) OR ( ( ( l[4][0] ) OR ( ( l[3][0] ) AND ( l[5][0] ) ) ) OR ( l[0][0] ) ) ) ) OR ( ( x[3] == 1 ) AND ( ( l[3][0] ) OR ( ( ( l[4][0] ) OR ( ( l[3][0] ) AND ( l[5][0] ) ) ) OR ( l[0][0] ) ) ) ) ) 

l[1][1]:
     ( ( l[1][0] ) OR ( x[0] == 1 ) OR ( ( x[1] == 1 ) AND ( l[0][0] ) ) ) 

l[2][1]:
     ( ( l[2][0] ) OR ( x[2] == 1 ) OR ( x[3] == 1 ) ) 

l[3][1]:
     ( ( l[3][0] ) OR ( ( x[2] == 1 ) AND ( ( l[0][0] ) OR ( ( ( l[4][0] ) OR ( ( l[3][0] ) AND ( l[5][0] ) ) ) AND ( ( l[5][0] ) OR ( ( l[4][0] ) AND ( l[3][0] ) ) ) ) ) ) OR ( ( x[3] == 1 ) AND ( ( l[0][0] ) OR ( ( ( l[4][0] ) OR ( ( l[3][0] ) AND ( l[5][0] ) ) ) AND ( ( l[5][0] ) OR ( ( l[4][0] ) AND ( l[3][0] ) ) ) ) ) ) ) 

l[4][1]:
     ( ( l[4][0] ) OR ( ( x[2] == 1 ) AND ( ( ( ( l[3][0] ) OR ( ( l[4][0] ) AND ( l[5][0] ) ) ) AND ( ( l[5][0] ) OR ( ( l[4][0] ) AND ( l[3][0] ) ) ) ) OR ( ( l[0][0] ) OR ( ( l[4][0] ) OR ( ( l[3][0] ) AND ( l[5][0] ) ) ) ) ) ) OR ( ( x[3] == 1 ) AND ( ( ( ( l[3][0] ) OR ( ( l[4][0] ) AND ( l[5][0] ) ) ) AND ( ( l[5][0] ) OR ( ( l[4][0] ) AND ( l[3][0] ) ) ) ) OR ( ( l[0][0] ) OR ( ( l[4][0] ) OR ( ( l[3][0] ) AND ( l[5][0] ) ) ) ) ) ) ) 

l[5][1]:
     ( ( l[5][0] ) OR ( ( x[2] == 1 ) AND ( ( ( l[4][0] ) OR ( ( l[3][0] ) AND ( l[5][0] ) ) ) AND ( ( l[3][0] ) OR ( ( l[4][0] ) AND ( l[5][0] ) ) ) ) ) OR ( ( x[3] == 1 ) AND ( ( ( l[4][0] ) OR ( ( l[3][0] ) AND ( l[5][0] ) ) ) AND ( ( l[3][0] ) OR ( ( l[4][0] ) AND ( l[5][0] ) ) ) ) ) ) 

l[0][2]:
     ( ( l[0][1] ) OR ( ( x[1] == 2 ) AND ( l[1][1] ) ) OR ( ( x[2] == 2 ) AND ( ( l[3][1] ) OR ( ( ( l[4][1] ) OR ( ( l[3][1] ) AND ( l[5][1] ) ) ) OR ( l[0][1] ) ) ) ) OR ( ( x[3] == 2 ) AND ( ( l[3][1] ) OR ( ( ( l[4][1] ) OR ( ( l[3][1] ) AND ( l[5][1] ) ) ) OR ( l[0][1] ) ) ) ) ) 

l[1][2]:
     ( ( l[1][1] ) OR ( x[0] == 2 ) OR ( ( x[1] == 2 ) AND ( l[0][1] ) ) ) 

l[2][2]:
     ( ( l[2][1] ) OR ( x[2] == 2 ) OR ( x[3] == 2 ) ) 

l[3][2]:
     ( ( l[3][1] ) OR ( ( x[2] == 2 ) AND ( ( l[0][1] ) OR ( ( ( l[4][1] ) OR ( ( l[3][1] ) AND ( l[5][1] ) ) ) AND ( ( l[5][1] ) OR ( ( l[4][1] ) AND ( l[3][1] ) ) ) ) ) ) OR ( ( x[3] == 2 ) AND ( ( l[0][1] ) OR ( ( ( l[4][1] ) OR ( ( l[3][1] ) AND ( l[5][1] ) ) ) AND ( ( l[5][1] ) OR ( ( l[4][1] ) AND ( l[3][1] ) ) ) ) ) ) ) 

l[4][2]:
     ( ( l[4][1] ) OR ( ( x[2] == 2 ) AND ( ( ( ( l[3][1] ) OR ( ( l[4][1] ) AND ( l[5][1] ) ) ) AND ( ( l[5][1] ) OR ( ( l[4][1] ) AND ( l[3][1] ) ) ) ) OR ( ( l[0][1] ) OR ( ( l[4][1] ) OR ( ( l[3][1] ) AND ( l[5][1] ) ) ) ) ) ) OR ( ( x[3] == 2 ) AND ( ( ( ( l[3][1] ) OR ( ( l[4][1] ) AND ( l[5][1] ) ) ) AND ( ( l[5][1] ) OR ( ( l[4][1] ) AND ( l[3][1] ) ) ) ) OR ( ( l[0][1] ) OR ( ( l[4][1] ) OR ( ( l[3][1] ) AND ( l[5][1] ) ) ) ) ) ) ) 

l[5][2]:
     ( ( l[5][1] ) OR ( ( x[2] == 2 ) AND ( ( ( l[4][1] ) OR ( ( l[3][1] ) AND ( l[5][1] ) ) ) AND ( ( l[3][1] ) OR ( ( l[4][1] ) AND ( l[5][1] ) ) ) ) ) OR ( ( x[3] == 2 ) AND ( ( ( l[4][1] ) OR ( ( l[3][1] ) AND ( l[5][1] ) ) ) AND ( ( l[3][1] ) OR ( ( l[4][1] ) AND ( l[5][1] ) ) ) ) ) ) 

l[0][3]:
     ( ( l[0][2] ) OR ( ( x[1] == 3 ) AND ( l[1][2] ) ) OR ( ( x[2] == 3 ) AND ( ( l[3][2] ) OR ( ( ( l[4][2] ) OR ( ( l[3][2] ) AND ( l[5][2] ) ) ) OR ( l[0][2] ) ) ) ) OR ( ( x[3] == 3 ) AND ( ( l[3][2] ) OR ( ( ( l[4][2] ) OR ( ( l[3][2] ) AND ( l[5][2] ) ) ) OR ( l[0][2] ) ) ) ) ) 

l[1][3]:
     ( ( l[1][2] ) OR ( x[0] == 3 ) OR ( ( x[1] == 3 ) AND ( l[0][2] ) ) ) 

l[2][3]:
     ( ( l[2][2] ) OR ( x[2] == 3 ) OR ( x[3] == 3 ) ) 

l[3][3]:
     ( ( l[3][2] ) OR ( ( x[2] == 3 ) AND ( ( l[0][2] ) OR ( ( ( l[4][2] ) OR ( ( l[3][2] ) AND ( l[5][2] ) ) ) AND ( ( l[5][2] ) OR ( ( l[4][2] ) AND ( l[3][2] ) ) ) ) ) ) OR ( ( x[3] == 3 ) AND ( ( l[0][2] ) OR ( ( ( l[4][2] ) OR ( ( l[3][2] ) AND ( l[5][2] ) ) ) AND ( ( l[5][2] ) OR ( ( l[4][2] ) AND ( l[3][2] ) ) ) ) ) ) ) 

l[4][3]:
     ( ( l[4][2] ) OR ( ( x[2] == 3 ) AND ( ( ( ( l[3][2] ) OR ( ( l[4][2] ) AND ( l[5][2] ) ) ) AND ( ( l[5][2] ) OR ( ( l[4][2] ) AND ( l[3][2] ) ) ) ) OR ( ( l[0][2] ) OR ( ( l[4][2] ) OR ( ( l[3][2] ) AND ( l[5][2] ) ) ) ) ) ) OR ( ( x[3] == 3 ) AND ( ( ( ( l[3][2] ) OR ( ( l[4][2] ) AND ( l[5][2] ) ) ) AND ( ( l[5][2] ) OR ( ( l[4][2] ) AND ( l[3][2] ) ) ) ) OR ( ( l[0][2] ) OR ( ( l[4][2] ) OR ( ( l[3][2] ) AND ( l[5][2] ) ) ) ) ) ) ) 

l[5][3]:
     ( ( l[5][2] ) OR ( ( x[2] == 3 ) AND ( ( ( l[4][2] ) OR ( ( l[3][2] ) AND ( l[5][2] ) ) ) AND ( ( l[3][2] ) OR ( ( l[4][2] ) AND ( l[5][2] ) ) ) ) ) OR ( ( x[3] == 3 ) AND ( ( ( l[4][2] ) OR ( ( l[3][2] ) AND ( l[5][2] ) ) ) AND ( ( l[3][2] ) OR ( ( l[4][2] ) AND ( l[5][2] ) ) ) ) ) )

SyntaxError: invalid syntax (770808594.py, line 1)

In [ ]:
vector_k = k(2,2,4,1)
vector_k = clean_vector(vector_k)

print_vector(vector_k)

k(2,2,4,1) 
